In [1]:
import qiskit
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams.update({'font.size': 16})  # enlarge matplotlib fonts

# Suppress warnings
import warnings
warnings.filterwarnings('ignore')

# Import qubit states Zero (|0>) and One (|1>), and Pauli operators (X, Y, Z)
from qiskit.opflow import Zero, One, I, X, Y, Z

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from numpy import linalg as LA
%matplotlib inline
plt.rcParams.update({'font.size': 16})  # enlarge matplotlib fonts

# Suppress warnings
import warnings
warnings.filterwarnings('ignore')

# Import qubit states Zero (|0>) and One (|1>), and Pauli operators (X, Y, Z)
from qiskit.opflow import Zero, One, I, X, Y, Z


In [3]:
# Importing standard Qiskit modules
from qiskit import QuantumCircuit, QuantumRegister, IBMQ, execute, transpile
from qiskit.providers.aer import QasmSimulator
from qiskit.tools.monitor import job_monitor
from qiskit.circuit import Parameter

# Import state tomography modules
from qiskit.ignis.verification.tomography import state_tomography_circuits, StateTomographyFitter
from qiskit.quantum_info import state_fidelity

# suppress warnings
import warnings
warnings.filterwarnings('ignore')

In [4]:
import itertools


# Defining the Hamiltonian

In [5]:
def u_t(H, t):
    return (t * H).exp_i()

In [6]:
# IBM code, Returns the matrix representation of the XXX Heisenberg model for 3 spin-1/2 particles in a line
def H_heis3():
    # Interactions (I is the identity matrix; X, Y, and Z are Pauli matricies; ^ is a tensor product)
    XXs = (I^X^X) + (X^X^I)
    YYs = (I^Y^Y) + (Y^Y^I)
    ZZs = (I^Z^Z) + (Z^Z^I)
    
    # Sum interactions
    H = XXs + YYs + ZZs
    
    # Return Hamiltonian
    return H

In [7]:
def Trotter_matrix(t, trotter_steps):

    trotter_matrix = (I^I^I).to_matrix()
    t_matrix = (I^I^I).to_matrix()
    
    for term in terms:
        j, i = term[0], int(term[1]) - 1
                
        if j == 'x':
            operator  = X^X
        elif j == 'y':
            operator  = Y^Y
        elif j  == 'z':
            operator  = Z^Z
            
        if i == 0:
            operator = (operator^I)
        else:
            operator = (I^operator)
        
        u_t_op = u_t(operator, t/trotter_steps).to_matrix()
        t_matrix = np.matmul(t_matrix, u_t_op)
        
    for _ in range(trotter_steps):
        trotter_matrix = np.matmul(trotter_matrix, t_matrix)
        
    return trotter_matrix

# Error Norm

In [8]:
def error(terms, t, trotter_steps):

    trotter_matrix =  Trotter_matrix(t, trotter_steps)
    error_matrix = u_t(H_heis3(), t).to_matrix() - trotter_matrix
    w, v = LA.eig(error_matrix)
    w_abs = np.abs(w)
    
    return max(w_abs)

In [9]:
terms = ['x1', 'y1', 'z1', 'x2', 'y2', 'z2']

In [10]:
error(terms, 1, 3)

0.5596150062889852

In [11]:
# enumeration over permutations
def find_best_order(t, trotter_steps):

    best_perm = terms
    min_error = 2
    errors = []

    for i in itertools.permutations(terms):  
        error_i = error(i, t, trotter_steps)
        errors.append(error_i)
        
        if min_error > error_i:
            min_error = error_i
            best_perm = i
            
    return best_perm, min_error, errors
        

In [ ]:
best_perm, min_error, errors = find_best_order(1, 10)

In [ ]:
best_perm

In [ ]:
min_error

In [ ]:
num_bins = 6
n, bins, patches = plt.hist(errors, num_bins, facecolor='blue', alpha=0.5)
plt.show()

In [ ]:
errors_naive = [error(terms, 1, n) for n in range(10, 30)]

In [ ]:
plt.scatter(range(10, 30), errors_naive)

In [ ]:
errors_ordered = []
best_perms = []
for n in range(10, 30):
    print('Starting ', n, " trotterization steps")
    best_perm, min_error, errors = find_best_order(1, n)
    best_perms.append(best_perm)

    errors_ordered.append(min_error)

In [ ]:
plt.scatter(range(10, 30), errors_ordered, c = 'orange')
plt.scatter(range(10, 30), errors_naive)

In [ ]:
best_perms

In [ ]:
theta = np.pi/6

qc = QuantumCircuit(2)
qc.cx(0, 1)
qc.crx(-1 * theta, 1, 0)
qc.cx(0, 1)

print(qc)

In [ ]:
from qiskit.quantum_info import Operator
Operator(qc).data

In [ ]:
qc.decompose()